# How Time Works

In the graph there is a node labled `time` with no edges. At the beginning `currentTime=0`, and is autoincremented by 1 indefinitely. Time is always running and never reset.

An Azure function `functions/time` runs all events and actions that require time to complete. That runs on an NCRONTAB job specified in the `function.json` doc. It can be adjusted to speed up or slow down time in the game. 

Example NCRONTAB configurations:
* `"0 */2 * * * *"`  Every two minutes
* `"0 */5 * * * *"`  Every five minutes

Note that some events are taken care of by `functions/cosmicevents` and not by time.


In [10]:
import sys, os
import pandas as pd
sys.path.append('..')

from app.connectors import cmdb_graph
from app.functions import maths
from app.objects import time

c = cmdb_graph.CosmosdbClient()


import  ssl, asyncio

ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
import nest_asyncio
# this is required for running in a Jupyter Notebook. 
nest_asyncio.apply()

In [11]:
time = time.Time(c)
time

< time at UTU:None >

The time object doesn't query the graph until it is initialized. 

In [12]:
time.get_current_UTU()
time

< time at UTU:78251 >

The process just takes the UTU, adds 1 and patches. 

In [13]:
time.global_ticker()

'currentTime was updated from:78251 to: 78252'

You can confirm that it was updated by re-running the `get_current_UTU`. This does not cause an update to the graph, but it does update the UTU in the time object. 

In [14]:
time.get_current_UTU()
time

< time at UTU:78252 >

In [15]:
Arbitrary break to keep from auto-running

SyntaxError: invalid syntax (1155785363.py, line 1)

# Setting time

In [17]:
time_query = f"g.addV('time').property('currentTime',1000).property('updated_at','{time.utc_timestamp}').property('label','time').property('objid','{maths.uuid()}')"
time_query


"g.addV('time').property('currentTime',1000).property('updated_at','2024-12-31T20:02:19.335310+00:00').property('label','time').property('objid','5074521528225')"

In [7]:
c.run_query(time_query)

## Validate what was created

In [8]:
c.run_query('g.V().hasLabel("time").valueMap()')
c.res

[{'currentTime': [1000],
  'updated_at': ['2023-09-04T19:39:57.653661+00:00'],
  'objid': ['3297686412899']}]

In [5]:
c.run_query('g.V().hasLabel("time").drop()')
c.res

[]